# Crypto Currency Price Data

This documentation shows how to get the crypto currency price data from [Crypto Compare](https://min-api.cryptocompare.com/documentation?key=Historical&cat=dataHistoday).

Note: Python 3.9 is required.

In [ ]:
import requests, time
from datetime import datetime, date
from pandas import DataFrame as df

DAILY_URL = "https://min-api.cryptocompare.com/data/v2/histoday"
LIMIT_QUERY = 2000

In [ ]:
# currency: BTC, ETH, ...
# to_currency: USD
# Return a List of Objects
def daily_request(currency, to_currency, to_ts, limit): 
    params = {
        "fsym": currency,
        "tsym": to_currency,
        "limit": limit,
        "toTs": to_ts
    }
    req = requests.get(DAILY_URL, params = params).json()
    return req["Data"]

In [ ]:
# currency: BTC, ETH, ...
# to_currency: USD
# Return a List of Objects
def get_daily_data(currency, to_currency, days_remaining, to_ts = int(datetime.now().timestamp())):
    result = []
    last_time_from = to_ts - 1

    while days_remaining > 0:
        days = min(LIMIT_QUERY, days_remaining-1)
        res = daily_request(currency, to_currency, last_time_from, days)
        data = res["Data"]
        result += data[::-1]
        days_remaining -= len(data)
        last_time_from = res["TimeFrom"]-1
    return result

In [ ]:
def add_date(list_data):
    return [data | {"date": date.fromtimestamp(data["time"]) } for data in list_data]

In [ ]:
def get_daily_data_df(currency, to_currency, days_remaining, to_ts = int(datetime.now().timestamp())):
    return df(add_date(get_daily_data(currency, to_currency, days_remaining, to_ts)))

In [ ]:
# Create a dataframe for the data
dataframe = get_daily_data_df("BTC", "USD", 10)
dataframe

In [ ]:
def save_df(dataframe, currency, to_currency, days, end_date_str, output_dir = "../raw_data"):
    name = "{}_{}_{}_{}.csv".format(currency, to_currency, days, end_date_str)
    dataframe.to_csv(output_dir + "/" + name)

In [ ]:
# save the dataframe to ./data
save_df(dataframe, "BTC", "USD", 10, "2022-03-03")

In [ ]:
# all_in_one
def get_and_save_daily_data(currency, to_currency, days, to_ts = int(datetime.now().timestamp()), output_dir = "../raw_data"):
    dataframe = get_daily_data_df(currency, to_currency, days, to_ts)
    end_date_str = date.fromtimestamp(to_ts).strftime("%Y-%m-%d")
    save_df(dataframe, currency, to_currency, days, end_date_str)
    print("done")

In [ ]:
# Example of all_in_one

get_and_save_daily_data("BTC", "USD", 4000)

In [ ]:
# list of currencies and list of to_currencies
list_currencies = [
    "BTC", "ETH", "USDT", "USDC", "XRP", "LUNA", "SOL", "ADA", "AVAX", "DOT",
    "DOGE", "UST", "SHIB", "WBTC", "MATIC", "CRO", "DAI", "ATOM", "LTC", "NEAR",
    "LINK", "TRX", "BCH", "LEO", "ALGO", "OKB", "XLM", "FTM", "HBAR", "UNI"
]

list_to_currencies = ["USD", "EUR", "CNY", "JPY", "GBP", "CHF"]

In [ ]:
for currency in list_currencies:
    for to_currency in list_to_currencies:
        print(currency, to_currency)
        get_and_save_daily_data(currency, to_currency, 5000)
        time.sleep(0.1)
        